In [1]:
import requests
import urllib.parse
from bs4 import BeautifulSoup

In [2]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://www.ejawi.net/result.php"

headers = CaseInsensitiveDict()
headers["authority"] = "www.ejawi.net"
headers["accept"] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
headers["accept-language"] = "en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7"
headers["cache-control"] = "no-cache"
headers["content-type"] = "application/x-www-form-urlencoded"
headers["cookie"] = "__utma=248434904.1854521706.1653239537.1653239537.1653239537.1; __utmc=248434904; __utmz=248434904.1653239537.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; __utmb=248434904.1.10.1653239537"
headers["origin"] = "https://www.ejawi.net"
headers["pragma"] = "no-cache"
headers["referer"] = "https://www.ejawi.net/converterV2.php?go=rumi"
headers["sec-ch-ua"] = '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"'
headers["sec-ch-ua-mobile"] = "?0"
headers["sec-ch-ua-platform"] = '"macOS"'
headers["sec-fetch-dest"] = "iframe"
headers["sec-fetch-mode"] = "navigate"
headers["sec-fetch-site"] = "same-origin"
headers["sec-fetch-user"] = "?1"
headers["upgrade-insecure-requests"] = "1"
headers["user-agent"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"

In [3]:
def get_jawi(string):
    data = f"phonText={string}&disable_direction=on&number=1"
    resp = requests.post(url, headers=headers, data=data)
    decoded = resp._content.decode()
    soup = BeautifulSoup(decoded, 'html.parser')
    return soup.find_all('td')[0].text

In [4]:
get_jawi('"hello"')

'"هيللو"'

In [5]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/wikipedia/dumping-wiki-20-july-2019.json

In [6]:
import json
import random

In [7]:
with open('dumping-wiki-20-july-2019.json') as fopen:
    data = json.load(fopen)
    
len(data)

1303844

In [8]:
def window(string, n = 4):
    splitted = string.split()
    return [' '.join(splitted[i: i + n]) for i in range(0, len(splitted), n)]

window(data[0])

['bahasa melayu tulisan jawi',
 'bhs mlyw ialah sejenis',
 'bahasa melayu polinesia di',
 'bawah keluarga bahasa austronesia',
 'yang telah digunakan di',
 'wilayah malaysia indonesia dan',
 'persekitarannya sejak melebihi tahun',
 'lalu walaupun']

In [9]:
import mp

In [10]:
windows = []
for i in range(20000):
    windows.extend(window(data[i], n = random.randint(2, 6)))

In [11]:
len(windows)

172834

In [12]:
import re
from unidecode import unidecode

_punctuation = "!'(),.:;? +"
_special = '-'
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
_numbers = '0123456789'
_small_letters = 'abcdefghijklmnopqrstuvwxyz'

SYMBOLS = (
    list(_special) + list(_punctuation) + list(_letters) + list(_numbers)
)

def cleaning(string):
    string = unidecode(string)
    string = ''.join([c if c in SYMBOLS else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string.lower()).strip()
    return string

In [13]:
from tqdm import tqdm

def loop(files):
    windows, _ = files
    results = []
    for d in tqdm(windows):
        try:
            t = cleaning(d)
            if len(t):
                r = get_jawi(t)
                if len(r):
                    results.append((t, r))
        except Exception as e:
            print(e)
    return results
        

In [ ]:
r = mp.multiprocessing(windows, loop, cores = 12)

 14%|█▎        | 1977/14402 [06:38<51:31,  4.02it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 20%|██        | 2949/14402 [09:45<44:00,  4.34it/s]  

In [ ]:
with open('wikipedia-windows.json', 'w') as fopen:
    json.dump(r, fopen)